In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"

device = "cuda:0"

In [2]:
import torch
import torch.onnx
import torch.nn as nn
import torch.nn.init as init
import torch.utils.model_zoo as model_zoo
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.models import build_network
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import os
import numpy as np
import torch
# # import onnx
import onnxruntime as ort
import torch.nn as nn

# from typing import Sequence, NamedTuple



In [3]:
# cfg_file = "./onnx_config.yaml"
cfg_file = "./pillar/config.yaml"
cfg_from_yaml_file(cfg_file, cfg)
if os.path.exists('./deploy_pillar_sfaw_3d_r')==False:
    os.mkdir('./deploy_pillar_sfaw_3d_r')
log_file = './deploy_pillar_sfaw_3d_r/log_trt.log'
logger = common_utils.create_logger(log_file, rank=0)
test_set, test_loader, sampler = build_dataloader(
    dataset_cfg=cfg.DATA_CONFIG,
    class_names=cfg.CLASS_NAMES,
    batch_size=1,
    dist=False, workers=8, logger=logger, training=False
)

2024-04-11 04:55:55,339   INFO  Loading Custom dataset.
2024-04-11 04:55:55,340   INFO  Total samples for CUSTOM dataset: 0


In [4]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=test_set)
# cfg

In [5]:
ckpt = "./pillar/model.pth"
model.load_params_from_file(filename=ckpt, logger=logger, to_cpu=False, pre_trained_path=None)
model.eval()
model.cuda()

2024-04-11 04:55:57,496   INFO  ==> Loading parameters from checkpoint ./pillar/model.pth to GPU
2024-04-11 04:55:57,618   INFO  ==> Checkpoint trained from version: pcdet+0.6.0+255db8f
2024-04-11 04:55:57,649   INFO  ==> Done (loaded 391/391)


CenterPoint(
  (vfe): DynamicVoxelVFE(
    (pfn_layers): ModuleList(
      (0): PFNLayerV2(
        (linear): Linear(in_features=10, out_features=96, bias=False)
        (norm): BatchNorm1d(96, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
      (1): PFNLayerV2(
        (linear): Linear(in_features=192, out_features=192, bias=False)
        (norm): BatchNorm1d(192, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (relu): ReLU()
      )
    )
  )
  (backbone_3d): DSVT(
    (input_layer): DSVTInputLayer(
      (posembed_layers): ModuleList(
        (0): ModuleList(
          (0-3): 4 x ModuleList(
            (0-1): 2 x PositionEmbeddingLearned(
              (position_embedding_head): Sequential(
                (0): Linear(in_features=2, out_features=192, bias=True)
                (1): BatchNorm1d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU(inplace=True)
       

In [6]:
# batch_dict = torch.load("./batch_dict.pth", map_location="cuda")
# inputs = batch_dict
# inputs

In [6]:
batch_dict = torch.load("/mnt/nas2/users/eslim/onnx/sfaw/input_dict.pth", map_location="cuda")

inputs = batch_dict
inputs

{'points': tensor([[ 0.0000,  0.8040,  0.7984, -1.4852,  0.0000],
         [ 0.0000,  0.8203,  0.7921, -1.4947,  0.0000],
         [ 0.0000,  0.8500,  0.8309, -1.4720,  0.0000],
         ...,
         [ 0.0000, -0.6915,  0.6910,  1.2054,  0.0000],
         [ 0.0000, -0.6724,  0.6628,  1.2318,  0.0000],
         [ 0.0000, -0.6677,  0.6675,  1.2318,  0.0000]], device='cuda:0'),
 'frame_id': array(['2024-03-09 04:57:20'], dtype='<U19'),
 'lidar_aug_matrix': tensor([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]], device='cuda:0'),
 'use_lead_xyz': tensor([1.], device='cuda:0'),
 'batch_size': 1}

In [7]:
inputs_vfe = model.vfe(inputs)
inputs_vfe


{'points': tensor([[ 0.0000,  0.8040,  0.7984, -1.4852,  0.0000],
         [ 0.0000,  0.8203,  0.7921, -1.4947,  0.0000],
         [ 0.0000,  0.8500,  0.8309, -1.4720,  0.0000],
         ...,
         [ 0.0000, -0.6915,  0.6910,  1.2054,  0.0000],
         [ 0.0000, -0.6724,  0.6628,  1.2318,  0.0000],
         [ 0.0000, -0.6677,  0.6675,  1.2318,  0.0000]], device='cuda:0'),
 'frame_id': array(['2024-03-09 04:57:20'], dtype='<U19'),
 'lidar_aug_matrix': tensor([[[1., 0., 0., 0.],
          [0., 1., 0., 0.],
          [0., 0., 1., 0.],
          [0., 0., 0., 1.]]], device='cuda:0'),
 'use_lead_xyz': tensor([1.], device='cuda:0'),
 'batch_size': 1,
 'pillar_features': tensor([[0.0000, 0.0458, 0.0000,  ..., 0.0303, 0.0000, 0.0000],
         [0.0000, 0.0686, 0.0000,  ..., 0.0079, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.5312, 0.4485, 0.0000,  ..., 0.0691, 0.0000, 0.5523],
         [0.0000, 0.4938, 0.0000,  ..., 0.0557, 0.0

In [8]:
# data_dict = {}
input_dict = {}
input_dict['batch_dict']={}
input_dict['batch_dict']['voxel_features'] = inputs_vfe['voxel_features']
input_dict['batch_dict']['use_lead_xyz'] = inputs_vfe['use_lead_xyz']
input_dict['batch_dict']['batch_size'] = torch.tensor(inputs_vfe['batch_size']).to(device)
input_dict['batch_dict']['pillar_features'] = inputs_vfe['pillar_features']
input_dict['batch_dict']['voxel_coords'] = inputs_vfe['voxel_coords']
# input_dict['batch_dict']['voxel_coords'] = inputs_vfe['voxel_coords']


# data_dict.update(inputs_bev)
# data_dict

In [9]:
with torch.no_grad():
    DSVT_3d_Backbone = model.backbone_3d
    # dsvtblocks_list = DSVT_Backbone.stage_0
    # layer_norms_list = DSVT_Backbone.residual_norm_stage_0
    # inputs = model.vfe(inputs)
    # voxel_info = DSVT_Backbone.input_layer(inputs)
    # set_voxel_inds_list = [[voxel_info[f'set_voxel_inds_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    # set_voxel_masks_list = [[voxel_info[f'set_voxel_mask_stage{s}_shift{i}'] for i in range(2)] for s in range(1)]
    # pos_embed_list = [[[voxel_info[f'pos_embed_stage{s}_block{b}_shift{i}'] for i in range(2)] for b in range(4)] for s in range(1)]



In [11]:
input_names = [
    'voxel_features',
    'use_lead_xyz',
    'batch_size',
    'pillar_features',
    'voxel_coords',
]
output_names = ["output",]

In [16]:
base_name = "./deploy_pillar_sfaw_3d_r/dsvt_3d"
ts_path = f"{base_name}.ts"
onnx_path = f"{base_name}.onnx"

# allptransblocktrt = AllDSVTBlocksTRT(dsvtblocks_list, layer_norms_list).eval().cuda()

torch.onnx.export(
    DSVT_3d_Backbone,
    # alldsvtblockstrt_inputs,
    input_dict,
    onnx_path, 
    input_names=input_names,
    output_names=output_names, 
    # dynamic_axes=dynamic_axes,
    opset_version=14,
)
# https://github.com/Haiyang-W/DSVT/issues/60

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 1 ERROR ========================
ERROR: missing-standard-symbolic-function
Exporting the operator 'aten::bincount' to ONNX opset version 14 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues.
None
<Set verbose=True to see more details>




UnsupportedOperatorError: Exporting the operator 'aten::bincount' to ONNX opset version 14 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues.

In [ ]:
# # test onnx
# ort_session = ort.InferenceSession(onnx_path)
# def to_numpy(tensor):
#     return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# # compute ONNX Runtime output prediction
# ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(pillar_features),
#               ort_session.get_inputs()[1].name: to_numpy(set_voxel_inds_list[0][0]),
#               ort_session.get_inputs()[2].name: to_numpy(set_voxel_inds_list[0][1]),
#               ort_session.get_inputs()[3].name: to_numpy(set_voxel_masks_list[0][0]),
#               ort_session.get_inputs()[4].name: to_numpy(set_voxel_masks_list[0][1]),
#               ort_session.get_inputs()[5].name: to_numpy(torch.stack([torch.stack(v, dim=0) for v in pos_embed_list[0]], dim=0)),}
# ort_outs = ort_session.run(None, ort_inputs)
# ort_outs[0].shape
